In [46]:
from ipywidgets import HBox, VBox, Checkbox, Button, Label, Text, Layout, Output
from ipywidgets import HTML as HTMLwidget
from IPython.display import display, HTML

In [47]:
checkboxes = {}
vetoFile = ''
faveFile = ''
vetoList = {}
faveList = {}

In [48]:
def parseFile(file):
    # makes a dictionary from a veto or fave list
    ret = {}
    try:
        f = open(file).readlines()
    except:
        return {}
    for line in f:
        tokList = line.split('|')
        cleanTokList = [t.strip() for t in tokList]
        if len(cleanTokList) > 1:
            ret[cleanTokList[0]] = cleanTokList[1]
        else:
            ret[cleanTokList[0]] = ''
    return ret

In [49]:
def readFiles(passVetoFile, passFaveFile, verbose=False):
    global vetoList, faveList
    vetoFile = passVetoFile
    faveFile = passFaveFile
    vetoList = parseFile(vetoFile)
    faveList = parseFile(faveFile)
    if verbose:
        print ('Veto List is', vetoList)
        print ('Fave List is', faveList)

In [50]:
def writeFiles(verbose=True):
    global vetoFile, faveFile
    
    vfile = open(vetoFile, 'w')
    # past vetos that we read from the file
    for k,v in vetoList.items():
        vfile.write(k + '|' + v + '\n')
        
    ffile = open(faveFile, 'w')
    # past faves that we read from the file
    for k,v in faveList.items():
        ffile.write(k + '|' + v + '\n')
        
    for k,vfr in checkboxes.items():
        (v,f,r) = vfr
        #print(k, v, f, r)
        if v.value:
            if verbose:
                print('%s in vetolist' % k)
            vfile.write(k + '|' + r.value + '\n')
        if f and f.value:
            if verbose:
                print('%s in favelist' % k)
            ffile.write(k + '|' + r.value + '\n')
    vfile.close()
    ffile.close()
    print('Choices saved')

In [51]:
view = Output(layout={'border': '1px solid black'})

@view.capture(clear_output=True)
def changed(b):
    (box, objectId) = b.owner.description.split()
    on = b.old
    (v, f, r) = checkboxes[objectId]
    if v.value and f.value:
        if box == 'veto' and v.value: f.value = False
        if box == 'fave' and f.value: v.value = False
    if on and r:
        print('ON %s %s comment %s' % (box, objectId, r.value))
    else:
        print('OFF %s %s' % (box, objectId))
    writeFiles()

In [52]:
def showCheckboxes(objectId, bottomHTML):
    global vetoList, faveList
    faveText = None
    vetoText = None
    if objectId in faveList:
        fvalue = True
        bottomHTML += '<font color="red"><b> -- Favourite: %s</b></font>' % faveList[objectId]
    else:
        fvalue = False

    #display(HTML(text))
        
    if objectId in vetoList:
        vetoText = vetoList[objectId]
        vvalue = True
    else:
        vvalue = False
        
    v = Checkbox(
        value = vvalue,
        description = 'veto ' + objectId,
        #layout=Layout(width='15%'),
        color='lime', # Colors can be set by name,
    background_color='#0022FF', # and also by color code.
    border_color='cyan'
    )
    v.observe(changed)

    if vvalue:
        checkboxes[objectId] = (v, None, None)
        display(HBox([HTMLwidget(text), v]))
        #display(v)
    else:
        f = Checkbox(
            value = fvalue,
            description = 'fave '  + objectId,
            #layout=Layout(width='15%'),
        )
        f.observe(changed)
    
        r = Text(
            value=faveText,
            #layout=Layout(width='40%'),
        )
        checkboxes[objectId] = (v,f, r)
        display(HTML(bottomHTML))
        display(HBox([Label(value='Add comment, THEN click veto/fave'), v, f, r]))

In [53]:
vetoFile      = 'vetoTest.txt'
faveFile      = 'faveTest.txt'

readFiles(vetoFile, faveFile)

for i in range(5):
    objectId = 'Object%d' % i
    print('-----------------------')
    if objectId in vetoList:
        text = '%s vetoed:%s' % (objectId, vetoList[objectId])
    else:
        text = '<font size=5>%s</font>' % objectId
        
    showCheckboxes(objectId, text)

-----------------------


-----------------------


-----------------------


-----------------------


-----------------------


In [54]:
view

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…